<img src="images/array_print.png" alt="Array Print" align="right"/>

# Array Print

<code>Array Print</code> takes a <code>.csv</code> containing a MITOMI-assayable library in a tabular format and returns a Scienion-compatible <code>.fld</code> file and a csv record of the array layout.

<br>

<div style="width:70px; height:auto; float:left; display:inline">Author:</div>
<div>Micah Olivas</div>
<div style="width:70px; height:auto; float:left; display:inline">Created:</div>
<div>Sept 27, 2021</div>
<div style="width:70px; height:auto; float:left; display:inline">Updated:</div>
<div>May 23, 2022</div>

In [4]:
import os
from ipyfilechooser import FileChooser

import bin.array_print_core as ap

## Import `.csv` containing samples in tabular format

In [5]:
# take user input and get current working directory
user_binning = str(input('Does your print require catalytic binning (specific to HT-MEK)? (yes or no)'))
cwd = os.getcwd()

if user_binning == 'yes':
    print('Select `.csv` containing library positions in tabular format')
    library_csv = FileChooser(cwd)
    display(library_csv)
    print('Select `.csv` containing catalytic bins of mutants in plate')
    fc_catalytic_bins = FileChooser(cwd)
    display(fc_catalytic_bins)
    # bin_desc = pd.read_csv('etc')
    
elif user_binning == 'no':
    print('Select `.csv` containing library positions in tabular format')
    library_csv = FileChooser(cwd)
    display(library_csv)

Does your print require catalytic binning (specific to HT-MEK)? (yes or no) no


Select `.csv` containing library positions in tabular format


FileChooser(path='/Users/duncanmuir/ucsf_biophysics/pinney_lab/Array-Print', filename='', title='', show_hidde…

In [6]:
library_df, column_names, library_members = ap.csv_to_df(library_csv)

,plate_number,plate_position,member_name,catalytic_bin
0,1,A1,gs_wt,2
1,1,A2,gs_i111l,2
2,1,A3,gs_q16f,4
3,1,A4,gs_r88g,4
4,1,A5,gs_p181a,3
5,1,A6,bs_wt,2
6,1,A7,bg_wt,1
7,1,A8,ec_wt,1


## Declare print dimensions
This script will maximize the number of sample replicates that appear in the final print given the size of your library, the print dimensions, and any space occuppied by skipped intervening rows. Additionally, you must declare how many blocks appear on your device and how many **empty columns**, if any, you'd like in your print.

In [10]:
# Number of columns and rows on device
total_columns = 56
total_rows = 32
empty_columns = 3
skip_rows = 'n'

library_members, library_size, empty_rows, columns, rows, replicates = ap.count_replicates(library_df, 
                                                                                           total_columns, 
                                                                                           total_rows, 
                                                                                           empty_columns, 
                                                                                           skip_rows)

Library contains 8 members. Accounting for skipped columns, the script will array 212 replicates per library member.


# Name file and generate array
**Change** the name of the file to include a description of your library and today's date. **BE AWARE THAT** each time you execute this cell, **the order of the array changes** and a new, timestamped `.csv` file is written to a project folder in the working directory.

In [11]:
filename = 'ADK_initial_muts_orths_20220602-1'
print_df, project_path = ap.generate_array(filename, library_df, total_columns, total_rows, skip_rows, column_names)

     0    1    2    3    4    5    6    7    8    9   ...   46   47   48   49  \
0   1A2  1A4  1A1  1A8  1A5  1A1  1A4  1A6  1A7  1A8  ...  1A6  1A4  1A3  1A8   
1   1A1  1A3  1A6  1A6  1A8  1A1  1A5  1A2  1A6  1A2  ...  1A6  1A3  1A7  1A3   
2   1A3  1A8  1A4  1A5  1A8  1A5  1A2  1A1  1A7  1A6  ...  1A1  1A1  1A1  1A3   
3   1A7  1A8  1A7  1A4  1A3  1A4  1A8  1A8  1A7  1A8  ...  1A3  1A2  1A8  1A3   
4   1A1  1A3  1A1  1A5  1A6  1A3  1A3  1A5  1A5  1A7  ...  1A1  1A1  1A5  1A4   
5   1A2  1A1  1A2  1A2  1A3  1A6  1A2  1A3  1A7  1A4  ...  1A2  1A2  1A8  1A7   
6   1A8  1A3  1A7  1A1  1A6  1A5  1A5  1A7  1A8  1A7  ...  1A4  1A4  1A3  1A2   
7   1A1  1A8  1A1  1A1  1A2  1A1  1A4  1A5  1A2  1A4  ...  1A8  1A4  1A1  1A5   
8   1A8  1A3  1A1  1A6  1A8  1A2  1A7  1A5  1A5  1A4  ...  1A1  1A1  1A4  1A3   
9   1A8  1A7  1A5  1A2  1A6  1A7  1A7  1A1  1A8  1A7  ...  1A2  1A4  1A1  1A3   
10  1A2  1A4  1A4  1A7  1A8  1A6  1A4  1A2  1A3  1A6  ...  1A8  1A5  1A3  1A4   
11  1A6  1A4  1A4  1A7  1A7 

## Output field file-formatted position list
Finally, the following cell writes a `.txt` file containing the array position list to the project folder. This text can be copied into a Scienion-compattible `.fld` file for printing. Note that a new, timestamped `.txt` file is written each time you execute this cell. 

In [12]:
ap.display_fld(print_df, total_columns, total_rows)
ap.write_fld(project_path, filename, print_df, total_columns, total_rows)

1/1	1A2,	1,
1/2	1A4,	1,
1/3	1A1,	1,
1/4	1A8,	1,
1/5	1A5,	1,
1/6	1A1,	1,
1/7	1A4,	1,
1/8	1A6,	1,
1/9	1A7,	1,
1/10	1A8,	1,
1/11	1A3,	1,
1/12	1A7,	1,
1/13	1A6,	1,
1/14	1A1,	1,
1/15	1A3,	1,
1/16	1A1,	1,
1/17	1A4,	1,
1/18	1A7,	1,
1/19	1A3,	1,
1/20	1A5,	1,
1/21	1A3,	1,
1/22	1A8,	1,
1/23	1A5,	1,
1/24	1A7,	1,
1/25	1A5,	1,
1/26	1A2,	1,
1/27	1A1,	1,
1/28	1A6,	1,
1/29	1A4,	1,
1/30	1A2,	1,
1/31	1A2,	1,
1/32	1A1,	1,
1/33	1A7,	1,
1/34	1A3,	1,
1/35	1A3,	1,
1/36	1A7,	1,
1/37	1A4,	1,
1/38	1A1,	1,
1/39	1A7,	1,
1/40	1A6,	1,
1/41	1A4,	1,
1/42	1A7,	1,
1/43	1A1,	1,
1/44	1A7,	1,
1/45	1A8,	1,
1/46	1A7,	1,
1/47	1A6,	1,
1/48	1A4,	1,
1/49	1A3,	1,
1/50	1A8,	1,
1/51	1A6,	1,
1/52	1A7,	1,
1/53	1A2,	1,
1/54	1A5,	1,
1/55	1A2,	1,
1/56	1A5,	1,
2/1	1A1,	1,
2/2	1A3,	1,
2/3	1A6,	1,
2/4	1A6,	1,
2/5	1A8,	1,
2/6	1A1,	1,
2/7	1A5,	1,
2/8	1A2,	1,
2/9	1A6,	1,
2/10	1A2,	1,
2/11	1A3,	1,
2/12	1A2,	1,
2/13	1A1,	1,
2/14	1A8,	1,
2/15	1A5,	1,
2/16	1A4,	1,
2/17	1A6,	1,
2/18	1A1,	1,
2/19	1A4,	1,
2/20	1A7,	1,
2/21	1A1,	1,
2/22	1A6,	1,
2/23